In [1]:
from IPython.lib.display import Audio
import re
import random
import pathlib
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
# !pip install voila
# !jupyter serverextension enable voila --sys-prefix

In [3]:
# !wget https://ndownloader.figshare.com/articles/11905533/versions/1 -O data.zip
# !unzip data.zip -d data
# !unzip data/AdultVocalizations.zip -d data/adult_vocalizations
# !unzip data/ChickVocalizations.zip -d data/chick_vocalizations

# !mkdir -p data/vocalizations
# !cp data/adult_vocalizations/* data/vocalizations/
# !cp data/chick_vocalizations/* data/vocalizations/

In [4]:
df = pd.read_csv('data/annotations.csv')

In [5]:
def create_expanded_button(description, button_style):
    return widgets.Button(
        description=description,
        button_style=button_style,
        layout=widgets.Layout(height='auto', width='auto')
    )

In [6]:
grid = widgets.GridspecLayout(5, 8)
call_types = sorted(df.call_type.unique())

out = widgets.Output(layout={'border': '1px solid black', 'margin': '20px auto'})

@out.capture(clear_output=True)
def update_output(button):
    lstrip = button.description.find('[')
    call_type = button.description[:lstrip-1]
    display(widgets.HTML(f'<p>Loaded call of type <strong>{call_type}</strong></p>'))
    fn = df[df.call_type == call_type].sample(1).fn.item()
    path = f'data/vocalizations/{fn}'

    with open(path, 'rb') as fd:
        contents = fd.read()
        audio = widgets.Audio(value=contents, autoplay=True, loop=False, controls=True)
    
    display(audio)

for i in range(5):
    for j in range(8):
        idx = i*8 + j
        if idx < len(call_types):
            n = df[df.call_type == call_types[idx]].shape[0]
            btn = create_expanded_button(f'{call_types[idx]} [{n}]', button_style='warning')
            btn.on_click(update_output)
            grid[i, j] = btn
    
initial_button = random.sample(grid.children, 1)[0] # initialize to some random value
update_output(initial_button)
    
widgets.VBox([widgets.HTML('<h3>Press to load a random recording with a given label</h3>'), grid, out])